In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv('extracted_phones.csv')
for col in df.columns:
    try:
        pd.to_numeric(df[col])
    except:
        pass

In [3]:
# df['release_year'] = df['release_year'].ffill()
# df['ip_rating'] = df['ip_rating'].fillna('IPXX')
df.loc[9065, 'price'] = 238.43
df.loc[7339, 'price'] = 1100

In [4]:
# Top 20 brands by average popularity
top_brands = df.groupby('brand')[['popularity_views']].mean('popularity_views').sort_values('popularity_views', ascending=False).reset_index().iloc[:20]['brand']
top_phones = df[df['brand'].isin(top_brands)]
top_phones.head()

,brand,model,photo_link,phone_link,popularity_become_fan,popularity_views,popularity_views_today,price,network_technology,eSIM,...,gps,nfc,radio,usb_type,usb_version,biometric_auth,has_black_color,foldable,battery_type,battery_capacity
738,Apple,iPad Air 13 (2025),https://fdn2.gsmarena.com/vv/bigpic/apple-ipad...,https://www.gsmarena.com/apple_ipad_air_13_(20...,16,44008,6.4,950.0,GSM / HSPA / LTE / 5G,True,...,True,False,False,USB Type-C,3.1,True,False,False,Li-Po,9705.0
739,Apple,iPad Air 11 (2025),https://fdn2.gsmarena.com/vv/bigpic/apple-ipad...,https://www.gsmarena.com/apple_ipad_air_11_(20...,19,48618,7.1,700.0,GSM / HSPA / LTE / 5G,True,...,True,False,False,USB Type-C,3.1,True,False,False,Li-Po,7606.0
740,Apple,iPad (2025),https://fdn2.gsmarena.com/vv/bigpic/apple-ipad...,https://www.gsmarena.com/apple_ipad_(2025)-137...,20,93684,16.0,400.0,GSM / HSPA / LTE / 5G,True,...,True,False,False,USB Type-C,2.0,True,False,False,Li-Po,NaN
741,Apple,iPhone 16e,https://fdn2.gsmarena.com/vv/bigpic/apple-ipho...,https://www.gsmarena.com/apple_iphone_16e-1339...,198,1895820,38.0,699.0,GSM / HSPA / LTE / 5G,True,...,True,True,False,USB Type-C,2.0,True,True,False,Li-Ion,4005.0
742,Apple,iPad mini (2024),https://fdn2.gsmarena.com/vv/bigpic/apple-ipad...,https://www.gsmarena.com/apple_ipad_mini_(2024...,70,535417,5.0,600.0,GSM / HSPA / LTE / 5G,True,...,True,False,False,USB Type-C,3.1,True,False,False,Li-Ion,5078.0


In [5]:
top_phones.loc[:, 'release_year'] = top_phones['release_year'].fillna(top_phones['announce_year'])
top_phones.loc[:, 'release_year'] = top_phones['release_year'].ffill()

In [6]:
new_phones = top_phones[top_phones['release_year'] >= 2020]

In [7]:
def label_device(row):
    model = row['model']
    brand = row['brand']
    
    # Check for tablet conditions
    if pd.notna(model) and ('pad' in model.lower() or 'tab' in model.lower()):
        return 'tablet'
    elif brand == 'HTC' and 'A10' in model:
        return 'tablet'
    elif brand == 'Lenovo' and 'Legion Y' in model:
        return 'tablet'
    elif brand == 'Lenovo' and 'M10' in model:
        return 'tablet'
    elif brand == 'Nokia' and 'T' in model:
        return 'tablet'
    # Check for watch condition
    elif pd.notna(model) and 'watch' in model.lower():
        return 'watch'
    else:
        return 'phone'  # Default to phone

In [8]:
# new_phones.loc[:, 'type'] = np.where(new_phones['model'].str.contains(r'pad|tab', case=False, na=False), 'tablet', pd.NA)
# new_phones.loc[(new_phones['brand'] == 'HTC') & (new_phones['model'].str.contains('A10', case=False, na=False)), 'type'] = 'tablet'
# new_phones.loc[(new_phones['brand'] == 'Lenovo') & (new_phones['model'].str.contains(r'Legion\s*Y', case=False, na=False)), 'type'] = 'tablet'
# new_phones.loc[(new_phones['brand'] == 'Lenovo') & (new_phones['model'].str.contains(r'M10', case=False, na=False)), 'type'] = 'tablet'
# new_phones.loc[(new_phones['brand'] == 'Nokia') & (new_phones['model'].str.contains(r'T\d+', case=False, na=False)), 'type'] = 'tablet'
# new_phones.loc[:, 'type'] = new_phones['type'].fillna(np.where(new_phones['model'].str.contains('Watch', case=False, na=False), 'watch', pd.NA))
# new_phones.loc[:, 'type'] = new_phones['type'].fillna('phone')

new_phones.loc[:, 'type'] = new_phones.apply(label_device, axis=1)

/tmp/ipykernel_119127/3977343561.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_phones.loc[:, 'type'] = new_phones.apply(label_device, axis=1)


In [9]:
new_phones['popularity_views_today'] = new_phones['popularity_views_today'].fillna(0)
avg_model_price = new_phones.groupby(['brand', new_phones['model'].str.split().str[0]])[['price']].mean().reset_index()
new_phones['5g'] = new_phones['network_technology'].str.contains(r'5g', case=False, na=False)
new_phones['ip_rating'] = new_phones['ip_rating'].fillna('IPXX')


/tmp/ipykernel_119127/2391138923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_phones['popularity_views_today'] = new_phones['popularity_views_today'].fillna(0)
/tmp/ipykernel_119127/2391138923.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_phones['5g'] = new_phones['network_technology'].str.contains(r'5g', case=False, na=False)
/tmp/ipykernel_119127/2391138923.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [10]:
new_phones.columns

Index(['brand', 'model', 'photo_link', 'phone_link', 'popularity_become_fan',
       'popularity_views', 'popularity_views_today', 'price',
       'network_technology', 'eSIM', 'announce_year', 'available',
       'release_year', 'cancelled', 'height_mm', 'length_mm', 'width_mm',
       'weight_g', 'ip_rating', 'screen_type', 'screen_hz', 'screen_size',
       'screen_to_body', 'screen_resolution_x', 'screen_resolution_y',
       'os_version', 'chipset_nm', 'chipset_cores', 'gpu_model',
       'memory_card_slot', 'internal_rom', 'internal_rom_unit', 'internal_ram',
       'internal_ram_unit', 'camera_mp', 'camera_f', 'camera_video_resolution',
       'camera_video_fps', 'selfie_camera_mp', 'selfie_camera_f',
       'selfie_camera_video_resolution', 'selfie_camera_video_fps',
       'loudspeaker', '35mm_jack', 'wifi_model', 'bluetooth_version', 'gps',
       'nfc', 'radio', 'usb_type', 'usb_version', 'biometric_auth',
       'has_black_color', 'foldable', 'battery_type', 'battery_capaci

In [11]:
relevant_features = pd.DataFrame()
relevant_features[['brand', 'model', 'photo_link', 'phone_link', 'popularity_become_fan',
       'popularity_views', 'popularity_views_today', 'price', 'eSIM', 'announce_year', 'available',
       'release_year', 'cancelled', 'height_mm', 'length_mm', 'width_mm',
       'weight_g', 'ip_rating', 'screen_type', 'screen_hz', 'screen_size',
       'screen_to_body', 'screen_resolution_x', 'screen_resolution_y', 'chipset_nm', 'chipset_cores', 
       'memory_card_slot', 'internal_rom', 'internal_rom_unit', 'internal_ram',
       'internal_ram_unit', 'camera_mp', 'camera_f', 'camera_video_resolution',
       'camera_video_fps', 'selfie_camera_mp', 'selfie_camera_f',
       'selfie_camera_video_resolution', 'selfie_camera_video_fps', '35mm_jack', 'wifi_model', 'bluetooth_version', 
       'nfc', 'radio', 'usb_type', 'usb_version', 'biometric_auth', 'has_black_color', 'foldable', 'battery_type', 'battery_capacity',
       'type', '5g']] = new_phones[['brand', 'model', 'photo_link', 'phone_link', 'popularity_become_fan',
       'popularity_views', 'popularity_views_today', 'price', 'eSIM', 'announce_year', 'available',
       'release_year', 'cancelled', 'height_mm', 'length_mm', 'width_mm',
       'weight_g', 'ip_rating', 'screen_type', 'screen_hz', 'screen_size',
       'screen_to_body', 'screen_resolution_x', 'screen_resolution_y', 'chipset_nm', 'chipset_cores', 
       'memory_card_slot', 'internal_rom', 'internal_rom_unit', 'internal_ram',
       'internal_ram_unit', 'camera_mp', 'camera_f', 'camera_video_resolution',
       'camera_video_fps', 'selfie_camera_mp', 'selfie_camera_f',
       'selfie_camera_video_resolution', 'selfie_camera_video_fps', '35mm_jack', 'wifi_model', 'bluetooth_version', 
       'nfc', 'radio', 'usb_type', 'usb_version', 'biometric_auth', 'has_black_color', 'foldable', 'battery_type', 'battery_capacity',
       'type', '5g']]

In [12]:
relevant_features.to_csv('relevant_features.csv', index=False)